In [9]:
from slackclient import SlackClient
import psycopg2 as pg
import pandas.io.sql as psql
#from datetime import datetime
import datetime
import sys
import json
import copy

print("Python version")
print (sys.version)
print("Version info.")
print (sys.version_info)
conn_str = "host={0} port={1} dbname={2} user={3} password={4}" \
.format("localhost", 5432, "kanjidb", "postgres", "w0lfpack")

try:
    conn = pg.connect(conn_str)
    print("Welcome to Jupyter Notebook.  You are connected to the Kanji  database!")
except pg.OperationalError:
    print("You are not connected to the database.")

Python version
3.7.2 (default, Dec 29 2018, 06:19:36) 
[GCC 7.3.0]
Version info.
sys.version_info(major=3, minor=7, micro=2, releaselevel='final', serial=0)
Welcome to Jupyter Notebook.  You are connected to the Kanji  database!


In [10]:
_STATUS_PING_FAILURE = 10001
_DEPLOY_STATE_MAINTENANCE = 10002
_REPORT_INTERVAL_MINS = 15
_ALERT_THRESHOLD_COUNT = 10

_CHANNEL_NAME = "meshio"
_BANNER_ALERT_MESSAGE = "Node Alert"
_BANNER_OK_MESSAGE = "Sensor Network"
_SLACK_EMOJI = ':warning:'

_LOCATION_ID = 10002
#_LOCATION_ID = sys.argv[1]

_MESH_NETWORK = "parmesh_01"

In [11]:
messagetemplate = "[\
   {\"type\": \"section\", \
		\"text\": { \
			\"type\": \"mrkdwn\", \
			\"text\": \"*<fakeLink.toUserProfiles.com|Iris / Zelda 1-1>*\\nTuesday, January 21 4:00-4:30pm\\nBuilding 2 - Havarti Cheese (3)\\n2 guests\" \
		}, \
		\"accessory\": { \
			\"type\": \"image\", \
			\"image_url\": \"https://api.slack.com/img/blocks/bkb_template_images/notifications.png\", \
			\"alt_text\": \"calendar thumbnail\" \
		} \
	} ]"

nodetemplate = "{\"type\": \"context\", \
		\"elements\": [ \
			{ \
				\"type\": \"image\", \
				\"image_url\": \"https://www.dropbox.com/s/o2gjgfsph2cxgds/thermometer.jpg?raw=1\", \
				\"alt_text\": \"notifications warning icon\" \
			}, \
			{ \
				\"type\": \"mrkdwn\", \
				\"text\": \"*Conflicts with Team Huddle: 4:15-4:30pm*\" \
			} \
		] \
	}"

dividertemplate = "{ \
		\"type\": \"divider\" \
	} "


#locationimageurl = 'https://www.dropbox.com/s/k8td4cmja7vs30r/building.jpg?raw=1'

In [12]:
blockmessage = json.loads(messagetemplate)
nodemessage  = json.loads(nodetemplate)
divider = json.loads(dividertemplate)
now = datetime.datetime.now()

query = "SELECT location.description, location.imageurl, customer.slacktoken FROM kanji_location location \
JOIN kanji_customer customer ON location.customer_id=customer.idcustomer \
WHERE idlocation={0}".format(_LOCATION_ID)

cur = conn.cursor()
cur.execute(query)
row = cur.fetchone()
location = row[0]
locationimageurl = row[1]
_SLACK_TOKEN = row[2]

if locationimageurl is None:
  locationimageurl = 'https://www.dropbox.com/s/k8td4cmja7vs30r/building.jpg?raw=1' 

query = "SELECT pinglog.node_id, node.name, node.friendlyname, node.deploystate_id, COUNT(*) AS count FROM kanji_pinglog pinglog \
JOIN kanji_node node ON pinglog.node_id=idnode \
WHERE pinglog.pingstate_id={0} AND pinglog.timestamp>= NOW() - INTERVAL '{1} minutes' \
AND node.location_id={2} \
GROUP BY (pinglog.node_id, node.name, node.friendlyname, node.deploystate_id) ORDER BY COUNT(*);".format(_STATUS_PING_FAILURE, _REPORT_INTERVAL_MINS, _LOCATION_ID)

print(query)

cur = conn.cursor()
cur.execute(query)
rows = cur.fetchall()

totalfailures = 0

if len(rows) > 0:
    reportnodes = []    
    print("The query returned the following:") 
    for row in rows:
        print(row)
        node = {}    
        node['id'] = row[0]
        node["name"] = row[1]
        node['friendlyname'] = row[2]
        if len(node['friendlyname'])==0:
          node['friendlyname'] = row[1]
        node['deploystate'] = row[3]
        node['count'] = row[4]
        node['deploystate'] = 0
        node['level'] = "Elevated"
        if int(node['count'])> 5:
          node['level'] = "Severe"
        elif int(node['count'])> 2:
          node['level'] = "High"
        if node['deploystate']==_DEPLOY_STATE_MAINTENANCE:
          node['level'] = "Maintenance"
        totalfailures += row[3]
        reportnodes.append(node)    

    # start key replacement
    # the troubling image
    print(blockmessage[0]["accessory"]["image_url"])
    blockmessage[0]["accessory"]["image_url"] = locationimageurl

    # *Porter Farms - Building 1  / 02:15 PM Monday, March 18, 2019 
    # The current building temperature is 69.2° F
    blockmessage[0]["text"]["text"] = "*{0}* \
    \nAt {1} \
    \n*Ping reports {2} no-reply failures in the period.*".format(location, now.strftime("%-I:%M %p %A, %B %e, %Y"), totalfailures)

    for reportnode in reportnodes:
        print("reportnodename={0}".format(reportnode["name"]))
        thisnodemessage = copy.deepcopy(nodemessage)
        alerticonurl = "https://www.dropbox.com/s/khe23ou4myd0w5q/elevated.png?raw=1"
        if int(node['count'])> 5: 
          alerticonurl = "https://www.dropbox.com/s/ofcln9cyuwtynh4/severe.png?raw=1"
        elif int(node['count'])> 2:
          alerticonurl = "https://www.dropbox.com/s/ulym44i7blkk2wt/high.png?raw=1"
        thisnodemessage["elements"][0]["image_url"] = alerticonurl
        thisnodemessage["elements"][1]["text"] = "{0} / {2} {1} failures".format(reportnode["friendlyname"],reportnode["count"],reportnode["level"])
        blockmessage.append(thisnodemessage.copy())  
    blockmessage.append(divider)
    
    sc = SlackClient(_SLACK_TOKEN)
    response = sc.api_call("chat.postMessage", channel=_CHANNEL_NAME, blocks=blockmessage)
    print(blockmessage) 
    if not 'ok' in response or not response['ok']:
      print("Error posting message to Slack channel")
      print(blockmessage)
      print(response)
    else:
      print("Ok posting message to Slack channel")
else:
    print("There are no ping failures in this period.")

SELECT pinglog.node_id, node.name, node.friendlyname, node.deploystate_id, COUNT(*) AS count FROM kanji_pinglog pinglog JOIN kanji_node node ON pinglog.node_id=idnode WHERE pinglog.pingstate_id=10001 AND pinglog.timestamp>= NOW() - INTERVAL '15 minutes' AND node.location_id=10002 GROUP BY (pinglog.node_id, node.name, node.friendlyname, node.deploystate_id) ORDER BY COUNT(*);
The query returned the following:
(10021, 'xenon_04', 'SO-Farrowing 1 Temperature', 10001, 10)
https://api.slack.com/img/blocks/bkb_template_images/notifications.png
reportnodename=xenon_04
[{'type': 'section', 'text': {'type': 'mrkdwn', 'text': '*Building 1*     \nAt 2:04 PM Thursday, August  1, 2019     \n*Ping reports 10001 no-reply failures in the period.*'}, 'accessory': {'type': 'image', 'image_url': 'https://www.dropbox.com/s/vlvnokujfx67zav/porter-farms.jpg?raw=1', 'alt_text': 'calendar thumbnail'}}, {'type': 'context', 'elements': [{'type': 'image', 'image_url': 'https://www.dropbox.com/s/ofcln9cyuwtynh4/s